In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost

In [3]:
df= pd.read_csv(r"C:\Users\TOMORI\Desktop\data science\data-science-class\Housing_dataset_train.csv")

In [4]:
df.head()

,ID,loc,title,bedroom,bathroom,parking_space,price
0,3583,Katsina,Semi-detached duplex,2.0,2.0,1.0,1149999.565
1,2748,Ondo,Apartment,NaN,2.0,4.0,1672416.689
2,9261,Ekiti,NaN,7.0,5.0,NaN,3364799.814
3,2224,Anambra,Detached duplex,5.0,2.0,4.0,2410306.756
4,10300,Kogi,Terrace duplex,NaN,5.0,6.0,2600700.898


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             14000 non-null  int64  
 1   loc            12187 non-null  object 
 2   title          12278 non-null  object 
 3   bedroom        12201 non-null  float64
 4   bathroom       12195 non-null  float64
 5   parking_space  12189 non-null  float64
 6   price          14000 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 765.8+ KB


In [6]:
df.isnull().sum()

ID                  0
loc              1813
title            1722
bedroom          1799
bathroom         1805
parking_space    1811
price               0
dtype: int64

In [7]:
df.describe()

,ID,bedroom,bathroom,parking_space,price
count,14000.000000,12201.000000,12195.000000,12189.000000,1.400000e+04
mean,4862.700357,4.308171,3.134235,3.169825,2.138082e+06
std,3818.348214,2.441165,2.035950,1.599415,1.083057e+06
min,0.000000,1.000000,1.000000,1.000000,4.319673e+05
25%,1672.750000,2.000000,1.000000,2.000000,1.393990e+06
50%,3527.000000,4.000000,2.000000,3.000000,1.895223e+06
75%,8011.250000,6.000000,5.000000,4.000000,2.586699e+06
max,12999.000000,9.000000,7.000000,6.000000,1.656849e+07


In [8]:
def missing_value(column_name):
    if df.dtypes[column_name]== 'O':
        T= df[column_name].mode()[0]
    else:
        T= df[column_name].median()
        
    df[column_name].fillna(T, inplace= True)
    
missing_value('loc')
missing_value('title')
missing_value('bedroom')
missing_value('bathroom')
missing_value('parking_space')

In [9]:
df.isnull().sum()

ID               0
loc              0
title            0
bedroom          0
bathroom         0
parking_space    0
price            0
dtype: int64

FEATURE SCALING

In [35]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
scaler= MinMaxScaler()

In [37]:
df0= df.iloc[:, 3:]
df0

,bedroom,bathroom,parking_space,price
0,2.0,2.0,1.0,1149999.565
1,4.0,2.0,4.0,1672416.689
2,7.0,5.0,3.0,3364799.814
3,5.0,2.0,4.0,2410306.756
4,4.0,5.0,6.0,2600700.898
...,...,...,...,...
13995,4.0,7.0,3.0,2367927.861
13996,4.0,7.0,5.0,2228516.471
13997,8.0,6.0,5.0,2406812.693
13998,4.0,6.0,1.0,3348918.718


In [38]:
df1= pd.DataFrame(scaler.fit_transform(df0), columns= df0.columns)
df1

,bedroom,bathroom,parking_space,price
0,0.125,0.166667,0.0,0.044497
1,0.375,0.166667,0.6,0.076872
2,0.750,0.666667,0.4,0.181751
3,0.500,0.166667,0.6,0.122600
4,0.375,0.666667,1.0,0.134399
...,...,...,...,...
13995,0.375,1.000000,0.4,0.119974
13996,0.375,1.000000,0.8,0.111334
13997,0.875,0.833333,0.8,0.122384
13998,0.375,0.833333,0.0,0.180767


LABEL ENCODING

In [10]:
from sklearn import preprocessing

In [11]:
label_encoder= preprocessing.LabelEncoder()

In [14]:
df['loc']= label_encoder.fit_transform(df['loc'])
df['title']= label_encoder.fit_transform(df['title'])

In [45]:
df2= df.iloc[:, :3]

In [68]:
#JOINING ALL THE ENCODED COLUMN WITH THE FEATURE SCALING COLUMN 
new_df=pd.concat([df2, df1], axis=1, join='inner')
new_df.head()

,ID,loc,title,bedroom,bathroom,parking_space,price
0,3583,19,7,0.125,0.166667,0.0,0.044497
1,2748,27,0,0.375,0.166667,0.6,0.076872
2,9261,12,4,0.750,0.666667,0.4,0.181751
3,2224,3,3,0.500,0.166667,0.6,0.122600
4,10300,21,8,0.375,0.666667,1.0,0.134399


In [51]:
new_df.isnull().sum()

ID               0
loc              0
title            0
bedroom          0
bathroom         0
parking_space    0
price            0
dtype: int64

In [52]:
new_df.drop('ID', axis=1, inplace=True)

In [55]:
X= new_df.iloc[:, :5]
y= new_df.iloc[:, 5]

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=.20, random_state=2, shuffle= True)

In [60]:
from sklearn.linear_model import LinearRegression

In [61]:
model= LinearRegression()

In [62]:
model.fit(X_train, y_train)

LinearRegression()

In [63]:
y_pred= model.predict(X_test)

In [65]:
from sklearn.metrics import mean_squared_error as mse

In [66]:
mse(y_test, y_pred)

0.003251349910345998